# Initial conditions

Initial conditions for a time wave generating a $D$ dimensional mathematical toy universe

$$D=3+1=4$$
$$r_{\phi}=\frac{1}{2}+\frac{\sqrt{D+1}}{2}$$
$$r_o = \frac{1}{D}$$
$$r_{\eta} = \frac{1}{2}+i\frac{\sqrt{D-1}}{2}=e^{\frac{i\pi}{3}}$$

$$t_o = D+3+3+3$$

$$n \geq 0 \in \mathbb{N}$$


$$r_n=\frac{n+1}{r_o}$$

$$\omega^{n+1}=e^{i\Omega(n+1)}$$


$$\Omega=\frac{2\pi}{T_o}$$
# Time Phase Representation
$$
\phi_n = r_n \omega^{n+1}=r_n  e^{i\Omega(n+1)} 
$$


$$
\phi_n  =\frac{n+1}{r_o} \cdot \exp\left( i \cdot \frac{r_o}{r_{\phi}} \cdot (n+1) \right)
$$

# Time Norm Representation
$$
\eta_n = r_n r_o r_{\eta} = (n+1) \cdot r_{\eta}
$$


$$r_{\eta} = \frac{1}{2}+i\frac{\sqrt{D-1}}{2}=e^{\frac{i\pi}{3}}$$

# Time Norm + Phase 
$$
\theta_n = (n+1) \cdot r_{\eta}^{ (n+1)\left(\frac{r_o}{r_{\phi}}\right)} 
$$

# Time Relation or Sync factor




$$T_o=4\pi(1+\sqrt{D+1})$$
$$\tau_n = \frac{\eta_n}{\phi_n}$$
$$
\tau_n = \frac{\eta_n}{\phi_n} = \frac{r_{\eta}r_o}{ \omega^{n+1}}=\frac{1}{4}e^{\frac{i\pi}{3}}e^{-i\Omega(n+1)}
$$

# Time Carrier Alpha 
$$
\alpha_n = \left( r_o + t_o - \frac{1}{2 r_o} (n+1) \right) \cdot \Re(\tau_n)
$$

# Time Carrier Beta 
$$
\beta_n = \left( r_o + t_o - \frac{\sqrt{D-1}}{2}(n+1) \right) \cdot \Im(\tau_n)
$$

# Identity monads carrier
$$
\gamma_n = \alpha_n + i \cdot \beta_n
$$


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib
matplotlib.use('Agg')
from matplotlib.colors import hsv_to_rgb

# Constants
D = 4  # 3+1 dimensions
r_phi = 1/2 + np.sqrt(D+1)/2
r_o = 1/D
r_eta = 1/2 + 1j * np.sqrt(D-1)/2
t_o = D + 3 + 3 + 3
T_o = 4 * np.pi * (1 + np.sqrt(D+1))
Omega = 2 * np.pi / T_o

# Calculate functions remain the same
def calculate_phi_n(n):
    return (n+1)/r_o * np.exp(1j * r_o/r_phi * (n+1))

def calculate_eta_n(n):
    return (n+1) * r_eta

def calculate_theta_n(n):
    return (n+1) * r_eta ** ((n+1)*(r_o/r_phi))

def calculate_tau_n(n):
    omega = np.exp(1j * Omega * (n+1))
    return r_eta * r_o / omega

def calculate_carriers(n):
    tau = calculate_tau_n(n)
    alpha = (r_o + t_o - 1/(2*r_o)*(n+1)) * np.real(tau)
    beta = (r_o + t_o - np.sqrt(D-1)/2*(n+1)) * np.imag(tau)
    gamma = alpha + 1j * beta
    return alpha, beta, gamma

def get_color(n):
    """Generate HSV color based on n mod t_o"""
    hue = (n % t_o) / t_o
    saturation = 1.0
    value = 1.0
    return hsv_to_rgb((hue, saturation, value))

def update_axis_limits(ax, x_data, y_data, padding=0.1):
    """Dynamically update axis limits with padding"""
    if len(x_data) > 0 and len(y_data) > 0:
        x_min, x_max = min(x_data), max(x_data)
        y_min, y_max = min(y_data), max(y_data)
        
        # Calculate ranges
        x_range = max(abs(x_max - x_min), 1e-10)
        y_range = max(abs(y_max - y_min), 1e-10)
        
        # Add padding
        x_pad = x_range * padding
        y_pad = y_range * padding
        
        # Set new limits
        ax.set_xlim(x_min - x_pad, x_max + x_pad)
        ax.set_ylim(y_min - y_pad, y_max + y_pad)
        
        # Make axes equal to maintain aspect ratio
        ax.set_aspect('equal', adjustable='box')

def create_2d_animation(calc_func, title, filename, n_points=100):
    fig, ax = plt.subplots(figsize=(10, 10), facecolor='black')
    ax.set_facecolor('black')
    
    # Style configurations
    plt.style.use('dark_background')
    ax.grid(True, color='dimgray', alpha=0.3)
    ax.set_title(title, color='white', pad=20)
    
    # Initialize empty lists for data
    x_data = []
    y_data = []
    colors = []
    scatter = ax.scatter([], [], c=[], s=50)
    
    # Line to connect points
    line, = ax.plot([], [], 'white', alpha=0.2, linewidth=0.5)
    
    def init():
        # Start with small initial window
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        return scatter, line

    def update(frame):
        value = calc_func(frame)
        x_data.append(np.real(value))
        y_data.append(np.imag(value))
        colors.append(get_color(frame))
        
        # Update scatter plot
        scatter.set_offsets(np.c_[x_data, y_data])
        scatter.set_color(colors)
        
        # Update line
        line.set_data(x_data, y_data)
        
        # Update axis limits dynamically
        update_axis_limits(ax, x_data, y_data, padding=0.2)
        
        # Add a timestamp
        ax.set_title(f'{title}\nStep: {frame}/{n_points-1}', color='white')
        
        # Ensure grid updates with new limits
        ax.grid(True, color='dimgray', alpha=0.3)
        
        return scatter, line

    anim = FuncAnimation(fig, update, frames=n_points,
                        init_func=init, blit=False,
                        interval=1000)  # 1 frame per second
    
    writer = FFMpegWriter(fps=4, metadata=dict(artist='Me'),
                         bitrate=1800)
    anim.save(filename, writer=writer)
    plt.close()

def create_all_animations():
    print("Creating animations...")
    
    # Time Phase
    print("Creating φₙ animation...")
    create_2d_animation(calculate_phi_n, 'Time Phase (φₙ)', 'time_phase_2d.mp4')
    
    # Time Norm
    print("Creating ηₙ animation...")
    create_2d_animation(calculate_eta_n, 'Time Norm (ηₙ)', 'time_norm_2d.mp4')
    
    # Time Norm + Phase
    print("Creating θₙ animation...")
    create_2d_animation(calculate_theta_n, 'Time Norm + Phase (θₙ)', 'theta_2d.mp4')
    
    # Time Relation
    print("Creating τₙ animation...")
    create_2d_animation(calculate_tau_n, 'Time Relation (τₙ)', 'tau_2d.mp4')
    
    # Identity Monads Carrier
    print("Creating γₙ animation...")
    create_2d_animation(lambda n: calculate_carriers(n)[2], 
                       'Identity Monads Carrier (γₙ)', 'gamma_2d.mp4')
    
    print("All animations complete!")

# Generate all animations
if __name__ == "__main__":
    create_all_animations()

Creating animations...
Creating φₙ animation...
Creating ηₙ animation...
Creating θₙ animation...
Creating τₙ animation...
Creating γₙ animation...
All animations complete!
